In [ ]:
# MS 434 Supply Chain Management Lab 3

# Group Number: 1
# Student Name: Senan Gaffori  
# Student ID: 20949022
# Student Name: Ayush Bhargava
# Student ID: 20889700

In [3]:
# Practice 1
import pandas as pd

def summarize_course(grades):
  return pd.Series({
        'Average': sum(grades) / len(grades),
        'Std Dev': pd.Series(grades).std(ddof=1),
        'Max': max(grades),
        'Min': min(grades)
    })

grades_MS251 = [78, 85, 92, 88, 76, 81, 90]
summary_MS251 = summarize_course(grades_MS251)
print("MS251 Summary:")
print(summary_MS251)

grades_MS434 = [84, 79, 91, 88, 75, 80, 85, 89]
summary_MS434 = summarize_course(grades_MS434)
print("MS434 Summary:")
print(summary_MS434)

grades_MS452 = [82, 87, 90, 85, 78, 80, 76, 88, 92]
summary_MS452 = summarize_course(grades_MS452)
print("MS452 Summary:")
print(summary_MS452)


MS251 Summary:
Average    84.285714
Std Dev     6.129554
Max        92.000000
Min        76.000000
dtype: float64
MS434 Summary:
Average    83.875000
Std Dev     5.514591
Max        91.000000
Min        75.000000
dtype: float64
MS452 Summary:
Average    84.222222
Std Dev     5.540257
Max        92.000000
Min        76.000000
dtype: float64


In [4]:
# Demonstration of "class", you don't need to do anything

class UserGymProfile:
    def __init__(self, name, age, gender, weight):
        self.name = name
        self.age = age
        self.gender = gender.lower()
        self.weight = weight

        # Advanced attributes: These will be set by run()
        self.age_group = None
        self.weight_category = None
        self.intensity = None
        self.schedule = {}

    def run(self):

        """Main controller: decide classification, intensity, schedule"""

        # Step 1: Basic functions: classify age and weight
        self.age_group = self.classify_age_group()
        self.weight_category = self.classify_weight_category()

        # Step 2: Analysis: decide intensity based on rules
        self.intensity = self.recommend_intensity()

        # Step 3: generate weekly schedule
        self.schedule = self.generate_schedule()

    def classify_age_group(self):

        if self.age < 18:
            return "teen"
        elif self.age <= 40:
            return "adult"
        else:
            return "senior"

    def classify_weight_category(self):

        if self.weight < 50:
            return "underweight"
        elif 50 <= self.weight <= 80:
            return "normal"
        else:
            return "overweight"

    def recommend_intensity(self):
        # Rule 1: Female and age ≥ 50 → generally lower intensity for joint and bone care
        if self.gender == "female" and self.age >= 50:
            return "low" if self.weight_category != "normal" else "moderate"

        # Rule 2: Teenagers
        if self.age_group == "teen":
            return "moderate" if self.weight_category == "normal" else "low"

        # Rule 3: Adults
        if self.age_group == "adult":
            if self.weight_category == "normal":
                return "high"
            elif self.weight_category == "underweight":
                return "moderate"
            else:  # overweight
                return "moderate"

        # Rule 4: Seniors
        if self.age_group == "senior":
            if self.weight_category == "normal":
                return "moderate"
            else:
                return "low"

        # Fallback
        return "moderate"

    def generate_schedule(self):
        if self.intensity == "low":
            return {
                "Monday": "Gentle walking or elliptical (30 min)",
                "Wednesday": "Light stretching / Yoga (30 min)",
                "Friday": "Chair-based exercises or swimming (30 min)",
            }
        elif self.intensity == "moderate":
            return {
                "Monday": "Cardio (bike or treadmill, 45 min)",
                "Wednesday": "Strength training (machines, 30 min)",
                "Friday": "Low-impact HIIT or circuit (30 min)",
            }
        else:  # high
            return {
                "Monday": "Strength training (upper body, 60 min)",
                "Tuesday": "Cardio + core (45 min)",
                "Thursday": "Strength training (lower body, 60 min)",
                "Saturday": "Full body HIIT (30 min)",
            }

    def show_summary(self):
        print(f"=== {self.name}'s Gym Plan ===")
        print(f"Age: {self.age} ({self.age_group})")
        print(f"Gender: {self.gender}")
        print(f"Weight: {self.weight}kg ({self.weight_category})")
        print(f"Recommended Intensity: {self.intensity}")
        print("Weekly Schedule:")
        for day, workout in self.schedule.items():
            print(f"  {day}: {workout}")

In [5]:
user1 = UserGymProfile(name = "Lily", age = 55, gender = "female", weight= 62)
user1.run()            # Runs logic and sets attributes
user1.show_summary()   # Displays results

# please test more cases on your own!

=== Lily's Gym Plan ===
Age: 55 (senior)
Gender: female
Weight: 62kg (normal)
Recommended Intensity: moderate
Weekly Schedule:
  Monday: Cardio (bike or treadmill, 45 min)
  Wednesday: Strength training (machines, 30 min)
  Friday: Low-impact HIIT or circuit (30 min)


In [ ]:
from docplex.mp.model import Model

class FlexibleProductMixOptimizer:
    def __init__(self, products, profits, usage, limits):
        """
        products: list of product names (e.g., ["A", "B", "C"])
        profits: dict of profit per unit (e.g., {"A": 30, "B": 20, "C": 40})
        usage: dict of resource usage (e.g., {"Machine": {"A": 2, "B": 3, "C": 1}})
        limits: dict of total available resources (e.g., {"Machine": 100})
        """
        self.products = products
        self.profits = profits
        self.usage = usage
        self.limits = limits
        self.model = Model(name="ProductMix")

    def run(self):
        self.build_model()
        return self.solve()

    def build_model(self):
        # Step 1: Define decision variables for each product
        self.product_vars = self.model.continuous_var_dict(self.products, name="Produce")

        # Step 2: Add constraints for each resource
        for resource, usage_per_product in self.usage.items():
            self.model.add_constraint(
                self.model.sum(
                    usage_per_product[prod] * self.product_vars[prod] for prod in self.products
                ) <= self.limits[resource],
                ctname=f"{resource}_limit"
            )

        # Step 3: Define the objective function (maximize total profit)
        self.model.maximize(
            self.model.sum(
                self.profits[prod] * self.product_vars[prod] for prod in self.products
            )
        )

    def solve(self):
        solution = self.model.solve()
        if solution:
            print("\nOptimal Production Plan")
            print("-----------------------")
            for prod in self.products:
                print(f"{prod}: {self.product_vars[prod].solution_value:.2f}")
            print(f"Total Profit: {self.model.objective_value:.2f}")
            return {prod: self.product_vars[prod].solution_value for prod in self.products}
        else:
            print("\nNo feasible solution found.")
            return None





Optimal Production Plan
-----------------------
A: 10.00
B: 0.00
C: 30.00
D: 0.00
Total Profit: 2100.00
{'A': 10.000000000000005, 'B': 0, 'C': 29.999999999999993, 'D': 0}


In [8]:
# products = ["A", "B"]
# profits = {"A": 30, "B": 35}
# usage = {"Machine": {"A": 1, "B": 2}, "Labor": {"A": 2, "B": 1}}
# limits = {"Machine": 100, "Labor": 80}

# products = ["A", "B", "C"]
# profits = {"A": 30, "B": 35, "C": 60}
# usage = {"Machine": {"A": 1, "B": 2, "C": 3}, "Labor": {"A": 2, "B": 1, "C": 2}}
# limits = {"Machine": 100, "Labor": 80}

products = ["A", "B", "C", "D"]
profits = {"A": 30, "B": 35, "C": 60, "D": 25}
usage = {"Machine": {"A": 1, "B": 2, "C": 3, "D": 1.5}, "Labor": {"A": 2, "B": 1, "C": 2, "D": 2}}
limits = { "Machine": 100, "Labor": 80}

optimizer = FlexibleProductMixOptimizer(products, profits, usage, limits)
result = optimizer.run()
# print(result)


Optimal Production Plan
-----------------------
A: 10.00
B: 0.00
C: 30.00
D: 0.00
Total Profit: 2100.00


In [9]:
# Heuristic for the VRP: Nearest neighber algorithm

import math
import pandas as pd
import numpy as np

# Re-define input data
data = [
    (0, 5, 5, 0),
    (1, 5, 8, 3),
    (2, 6, 7, 4),
    (3, 7, 9, 2),
    (4, 8, 5, 6),
    (5, 7, 3, 5),
    (6, 5, 3, 3),
    (7, 3, 2, 4),
    (8, 2, 5, 6)]
df = pd.DataFrame(data, columns=['node', 'x', 'y', 'demand'])
Q = 15  # truck capacity

# Step 1: Distance matrix
def calculate_distance_matrix(df):
    n = len(df)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i][j] = abs(df.loc[i, 'x'] - df.loc[j, 'x']) + abs(df.loc[i, 'y'] - df.loc[j, 'y']) 
    return pd.DataFrame(dist)

# Step 2: Nearest Neighbor algorithm
def nearest_neighbor(df, dist_matrix, capacity_limit):
    n = len(df)
    visited = [False] * n
    visited[0] = True  # depot is always visited
    routes = []
    total_distance = 0

    while not all(visited):
        current_route = [0]
        current_capacity = 0
        current_node = 0

        while True:
            nearest_node = None
            nearest_dist = float('inf')

            for i in range(1, n):
                if not visited[i] and current_capacity + df.loc[i, 'demand'] <= capacity_limit:
                    if dist_matrix.loc[current_node, i] < nearest_dist:
                        nearest_node = i
                        nearest_dist = dist_matrix.loc[current_node, i]

            if nearest_node is None:
                break

            visited[nearest_node] = True
            current_route.append(nearest_node)
            current_capacity += df.loc[nearest_node, 'demand']
            total_distance += nearest_dist
            current_node = nearest_node

        # Return to depot
        total_distance += dist_matrix.loc[current_node, 0]
        current_route.append(0)
        routes.append(current_route)

    return routes, total_distance

# Run the algorithm
dist_matrix = calculate_distance_matrix(df)
routes, total_dist = nearest_neighbor(df, dist_matrix, Q)

# Prepare output
route_output = []
for i, r in enumerate(routes, 1):
    route_output.append(f"Route {i}: {' - '.join(map(str, r))}")
route_output.append(f"Total distance: {total_dist:.2f}")
route_output


['Route 1: 0 - 6 - 5 - 4 - 0',
 'Route 2: 0 - 1 - 2 - 3 - 8 - 0',
 'Route 3: 0 - 7 - 0',
 'Total distance: 40.00']

In [11]:
#Assignment 3
import pandas as pd
import numpy as np
import math

class NearestNeighborAlgorithm:
    def __init__(self, filename, distance="manhattan", capacity_limit=15):
        """
        Parameters:
        - filename: Excel file path with columns [node, x, y, demand]
        - distance: 'manhattan' or 'euclidean'
        - capacity_limit: vehicle capacity
        """
        self.filename = filename
        self.distance_type = distance.lower()
        self.capacity = capacity_limit
        self.df = pd.read_excel(self.filename)
        self.n = len(self.df)
        self.dist_matrix = self._calculate_distance_matrix()

    def _calculate_distance_matrix(self):
        dist = np.zeros((self.n, self.n))
        for i in range(self.n):
            for j in range(self.n):
                x1, y1 = self.df.loc[i, 'x'], self.df.loc[i, 'y']
                x2, y2 = self.df.loc[j, 'x'], self.df.loc[j, 'y']
                if self.distance_type == "manhattan":
                    dist[i][j] = abs(x1 - x2) + abs(y1 - y2)
                elif self.distance_type == "euclidean":
                    dist[i][j] = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                else:
                    raise ValueError("Invalid distance type. Use 'manhattan' or 'euclidean'.")
        return pd.DataFrame(dist)

    def run(self):
        visited = [False] * self.n
        visited[0] = True  # depot
        routes = []
        total_distance = 0

        while not all(visited):
            current_route = [0]
            current_capacity = 0
            current_node = 0

            while True:
                nearest_node = None
                nearest_dist = float('inf')

                for i in range(1, self.n):
                    if not visited[i] and current_capacity + self.df.loc[i, 'demand'] <= self.capacity:
                        if self.dist_matrix.loc[current_node, i] < nearest_dist:
                            nearest_node = i
                            nearest_dist = self.dist_matrix.loc[current_node, i]

                if nearest_node is None:
                    break

                visited[nearest_node] = True
                current_route.append(nearest_node)
                current_capacity += self.df.loc[nearest_node, 'demand']
                total_distance += nearest_dist
                current_node = nearest_node

            # return to depot
            total_distance += self.dist_matrix.loc[current_node, 0]
            current_route.append(0)
            routes.append(current_route)

        return {
            "routes": routes,
            "total_distance": round(total_distance, 2)
        }
    
# HOW TO RUN
#  Assuming you have an Excel file named 'nodes.xlsx' with columns: node, x, y, demand
# group_0 = NearestNeighborAlgorithm("nodes.xlsx", distance="euclidean", capacity_limit=20)
# result = group_0.run()

#  Display result
# for i, route in enumerate(result["routes"], 1):
#     print(f"Route {i}: {' - '.join(map(str, route))}")
# print(f"Total distance: {result['total_distance']}")
